In [1]:
import os 
import datetime
from pathlib import Path
from dotenv import load_dotenv, find_dotenv

basepath = Path(os.getcwd())
# make sure your working directory is the repository root.
if basepath.name != "idp-radio-1":
    os.chdir(basepath.parent.parent.parent)
load_dotenv(find_dotenv())

%load_ext autoreload
%autoreload 2
os.getcwd()

'/srv/idp-radio-1'

In [2]:
import os 
import tensorflow as tf
from pathlib import Path
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.densenet import DenseNet121
from tensorflow.keras.applications.resnet_v2 import ResNet152V2
from src.architectures.simple.simple_base import SimpleBase
from src.architectures.benchmarks.benchmark import Benchmark, Experiment
from src.architectures.adv.guendel19 import densenet
from src.metrics.metrics import F2Score

Using TensorFlow backend.


In [3]:
# Specify which GPU(s) to use
os.environ["CUDA_VISIBLE_DEVICES"] = "1"  # Or 2, 3, etc. other than 0

config = tf.compat.v1.ConfigProto(allow_soft_placement=True, log_device_placement=True)
config.gpu_options.allow_growth = True
tf.compat.v1.Session(config=config)


Device mapping:
/job:localhost/replica:0/task:0/device:XLA_CPU:0 -> device: XLA_CPU device
/job:localhost/replica:0/task:0/device:XLA_GPU:0 -> device: XLA_GPU device
/job:localhost/replica:0/task:0/device:XLA_GPU:1 -> device: XLA_GPU device
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: GeForce RTX 2080 Ti, pci bus id: 0000:08:00.0, compute capability: 7.5
/job:localhost/replica:0/task:0/device:GPU:1 -> device: 1, name: GeForce RTX 2080 Ti, pci bus id: 0000:42:00.0, compute capability: 7.5



In [4]:
!remote_access/get_tunnels.sh

URLs open for the following services:
[('tensorboard', 'http://e703f1318d95.ngrok.io'), ('jupyternotebook', 'http://63ad65c6873d.ngrok.io'), ('jupyterlab', 'http://cf6f79db15c5.ngrok.io')]


In [5]:
metrics = [tf.keras.metrics.AUC(name="AUC_ROC", multi_label=True),
           tf.keras.metrics.Precision(),
           tf.keras.metrics.Recall(),
           F2Score(),
           tf.keras.metrics.BinaryAccuracy()
          ]

In [6]:

chexpert_columns = ['No Finding', 'Enlarged Cardiomediastinum', 'Cardiomegaly', 'Lung Opacity', 'Lung Lesion',
                       'Edema', 'Consolidation', 'Pneumonia', 'Atelectasis', 'Pneumothorax',
                       'Pleural Effusion', 'Pleural Other', 'Fracture', 'Support Devices']

chexpert_benchmark = Benchmark(Path(os.environ.get("CHEXPERT_DATASET_DIRECTORY")),
                                  chexpert_columns,
                                  epochs=10,
                                  train_labels="train.csv",
                                  path_column="Path",
                                  split_group='patient_id',
                                  batch_size=128,
                                  metrics = metrics)
chexpert_benchmark.as_dict()

{'dataset_name': 'chexpert_full',
 'dataset_folder': 'data/chexpert/full',
 'models_dir': 'models',
 'epochs': 10,
 'optimizer': 'Adam',
 'loss': 'binary_crossentropy',
 'metrics': ['AUC', 'Precision', 'Recall', 'F2Score', 'BinaryAccuracy'],
 'label_columns': ['No Finding',
  'Enlarged Cardiomediastinum',
  'Cardiomegaly',
  'Lung Opacity',
  'Lung Lesion',
  'Edema',
  'Consolidation',
  'Pneumonia',
  'Atelectasis',
  'Pneumothorax',
  'Pleural Effusion',
  'Pleural Other',
  'Fracture',
  'Support Devices'],
 'path_column': 'Path',
 'path_column_prefix': '',
 'shuffle': True,
 'batch_size': 128,
 'dim': (256, 256),
 'n_channels': 3,
 'nan_replacement': 0,
 'unc_value': -1,
 'u_enc': 'uzeroes',
 'drop_last': True,
 'train_num_samples': 142727,
 'valid_num_samples': 35855,
 'test_num_samples': 44832}

In [7]:
chestxray14_columns = ['Edema', 'Atelectasis', 'Pneumonia',
                       'Pleural_Thickening', 'Cardiomegaly', 'Infiltration', 'Consolidation',
                       'Fibrosis', 'No Finding', 'Effusion', 'Nodule', 'Mass', 'Hernia',
                       'Emphysema', 'Pneumothorax']

chestxray14_benchmark = Benchmark(Path(os.environ.get("CHESTXRAY14_DATASET_DIRECTORY")),
                                  chestxray14_columns,
                                  epochs=10,
                                  train_labels="meta/data/labels.csv",
                                  path_column="Image Index",
                                  path_column_prefix="images/",
                                  split_group='Patient ID',
                                  batch_size=128,
                                  metrics=metrics)
chestxray14_benchmark.as_dict()

{'dataset_name': 'chestxray14_scale_256',
 'dataset_folder': 'data/chestxray14/scale_256',
 'models_dir': 'models',
 'epochs': 10,
 'optimizer': 'Adam',
 'loss': 'binary_crossentropy',
 'metrics': ['AUC', 'Precision', 'Recall', 'F2Score', 'BinaryAccuracy'],
 'label_columns': ['Edema',
  'Atelectasis',
  'Pneumonia',
  'Pleural_Thickening',
  'Cardiomegaly',
  'Infiltration',
  'Consolidation',
  'Fibrosis',
  'No Finding',
  'Effusion',
  'Nodule',
  'Mass',
  'Hernia',
  'Emphysema',
  'Pneumothorax'],
 'path_column': 'Image Index',
 'path_column_prefix': 'images/',
 'shuffle': True,
 'batch_size': 128,
 'dim': (256, 256),
 'n_channels': 3,
 'nan_replacement': 0,
 'unc_value': -1,
 'u_enc': 'uzeroes',
 'drop_last': True,
 'train_num_samples': 72774,
 'valid_num_samples': 17487,
 'test_num_samples': 21859}

In [8]:
model_resnet_chestxray14 = SimpleBase(ResNet152V2, len(chestxray14_columns)).get_model()
model_resnet_chexpert = SimpleBase(ResNet152V2, len(chexpert_columns)).get_model()

model_densenet_chestxray14 = SimpleBase(DenseNet121, len(chestxray14_columns)).get_model()
model_densenet_chexpert = SimpleBase(DenseNet121, len(chexpert_columns)).get_model()

model_inceptionnet_chestxray14 = SimpleBase(InceptionV3, len(chestxray14_columns)).get_model()
model_inceptionnet_chexpert = SimpleBase(InceptionV3, len(chexpert_columns)).get_model()

In [9]:
experiment_resnet_chexpert = Experiment(chexpert_benchmark, model_resnet_chexpert, "ResNet152V2_CheXpert")
experiment_densenet_chexpert = Experiment(chexpert_benchmark, model_densenet_chexpert, "DenseNet121_CheXpert")
experiment_inceptionnet_chexpert = Experiment(chexpert_benchmark, model_inceptionnet_chexpert, "InceptionV3_CheXpert")

In [10]:
experiment_resnet_chestxray14 = Experiment(chestxray14_benchmark, model_resnet_chestxray14, "ResNet152V2_ChestXray14")
experiment_densenet_chestxray14 = Experiment(chestxray14_benchmark, model_densenet_chestxray14, "DenseNet121_ChestXray14")
experiment_inceptionnet_chestxray14 = Experiment(chestxray14_benchmark, model_inceptionnet_chestxray14, "InceptionV3_ChestXray14")

In [ ]:
experiment_resnet_chexpert_result =  experiment_resnet_chexpert.run()

Epoch 1/10
   1/1115 [..............................] - ETA: 0s - loss: 0.7224 - AUC_ROC: 0.5060 - precision: 0.1226 - recall: 0.3369 - f2_score: 0.2496 - binary_accuracy: 0.5212WARNING:tensorflow:From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
 372/1115 [=========>....................] - ETA: 8:32 - loss: 0.3279 - AUC_ROC: 0.6637 - precision: 0.6327 - recall: 0.4055 - f2_score: 0.4369 - binary_accuracy: 0.8629

In [ ]:
experiment_densenet_chexpert_result =  experiment_densenet_chexpert.run()

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 281 steps, validate for 68 steps
Epoch 1/5
  5/281 [..............................] - ETA: 38:58 - loss: 3.3168 - categorical_crossentropy: 3.3168 - categorical_accuracy: 0.2664 - categorical_hinge: 0.9445

Process Keras_worker_ForkPoolWorker-1:
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.6/multiprocessing/pool.py", line 125, in worker
    put((job, i, result))
  File "/usr/lib/python3.6/multiprocessing/queues.py", line 347, in put
    self._writer.send_bytes(obj)
  File "/usr/lib/python3.6/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/usr/lib/python3.6/multiprocessing/connection.py", line 398, in _send_bytes
    self._send(buf)
  File "/usr/lib/python3.6/multiprocessing/connection.py", line 368, in _send
    n = write(self._handle, buf)
KeyboardInterrupt
Exception in thread Thread-8:
Traceback (most recent call last):
  File "/usr/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.r

In [ ]:
experiment_inceptionnet_chexpert_result =  experiment_inceptionnet_chexpert.run()

In [ ]:
experiment_resnet_chestxray14_result =  experiment_resnet_chestxray14.run()

In [ ]:
experiment_densenet_chestxray14_result =  experiment_densenet_chestxray14.run()

In [ ]:
experiment_inceptionnet_chestxray14_result =  experiment_inceptionnet_chestxray14.run()

In [ ]:
model_guendel_chestxray14 = densenet(classes=len(chestxray14_columns))
model_guendel_chexpert = densenet(classes=len(chexpert_columns))

experiment_guendel_chestxray14 = Experiment(chestxray14_benchmark, model_guendel_chestxray14, "Guendel19_ChestXray14")
experiment_guendel_chexpert = Experiment(chexpert_benchmark, model_resnet_chexpert, "Guendel19_CheXpert")



In [ ]:
experiment_guendel_chestxray14_result =  experiment_guendel_chestxray14.run()

In [ ]:
experiment_guendel_chexpert_result =  experiment_guendel_chexpert.run()

In [14]:
train_result, eval_result, save_result = exp_result_resnet_chexpert
print(eval_result["report"])
print(eval_result["metrics"])

                            precision    recall  f1-score   support

                No Finding       0.20      0.67      0.30      4484
Enlarged Cardiomediastinum       0.00      0.00      0.00      2110
              Cardiomegaly       0.27      0.12      0.16      5353
              Lung Opacity       0.58      0.44      0.50     21193
               Lung Lesion       0.00      0.00      0.00      1769
                     Edema       0.45      0.43      0.44     10323
             Consolidation       0.00      0.00      0.00      2897
                 Pneumonia       0.00      0.00      0.00      1183
               Atelectasis       0.28      0.00      0.01      6797
              Pneumothorax       0.12      0.00      0.00      4044
          Pleural Effusion       0.55      0.76      0.64     17355
             Pleural Other       0.00      0.00      0.00       687
                  Fracture       0.24      0.01      0.02      1783
           Support Devices       0.54      0.98

In [14]:
valgen = experiment_resnet_chexpert.benchmark.valgen

valgen.on_epoch_end()

predictions = experiment_resnet_chexpert.model.predict(valgen, steps=len(valgen), verbose=1)


NameError: name 'experiment_resnet_chexpert' is not defined

In [35]:
import numpy as np
predictions_bool = (predictions >= 0.5)

y_pred = np.array(predictions_bool, dtype=int)

groundtruth_label = valgen.get_encoded_labels()
len(groundtruth_label[0])


['No Finding',
 'Enlarged Cardiomediastinum',
 'Cardiomegaly',
 'Lung Opacity',
 'Lung Lesion',
 'Edema',
 'Consolidation',
 'Pneumonia',
 'Atelectasis',
 'Pneumothorax',
 'Pleural Effusion',
 'Pleural Other',
 'Fracture',
 'Support Devices']

In [44]:
import sklearn
cm = sklearn.metrics.multilabel_confusion_matrix(groundtruth_label, y_pred)
for i in range(14):
    print(experiment_resnet_chexpert.benchmark.label_columns[i])
    print("true negatives: ", cm[i, 0,0])
    print("false negatives: ", cm[i, 1,0])
    print("true positives: ", cm[i, 1,1])
    print("false positives: ", cm[i, 0,1])
    print()

No Finding
true negatives:  22072
false negatives:  1073
true positives:  2268
false positives:  10171

Enlarged Cardiomediastinum
true negatives:  33764
false negatives:  1820
true positives:  0
false positives:  0

Cardiomegaly
true negatives:  29913
false negatives:  3883
true positives:  495
false positives:  1293

Lung Opacity
true negatives:  13091
false negatives:  9549
true positives:  7567
false positives:  5377

Lung Lesion
true negatives:  34120
false negatives:  1463
true positives:  1
false positives:  0

Edema
true negatives:  23048
false negatives:  4760
true positives:  3621
false positives:  4155

Consolidation
true negatives:  33197
false negatives:  2387
true positives:  0
false positives:  0

Pneumonia
true negatives:  34633
false negatives:  951
true positives:  0
false positives:  0

Atelectasis
true negatives:  30317
false negatives:  5193
true positives:  15
false positives:  59

Pneumothorax
true negatives:  32315
false negatives:  3266
true positives:  1
false

In [39]:

from sklearn.metrics import classification_report
report = classification_report(groundtruth_label, y_pred, target_names=list(resnet_chestxray.benchmark.label_columns))
print(report)

              precision    recall  f1-score   support

       Edema       0.03      0.55      0.06       432
 Atelectasis       0.11      0.82      0.19      2368
   Pneumonia       0.02      0.41      0.03       277
   Emphysema       0.02      0.00      0.01       456

   micro avg       0.07      0.65      0.13      3533
   macro avg       0.04      0.45      0.07      3533
weighted avg       0.08      0.65      0.14      3533
 samples avg       0.08      0.10      0.09      3533



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [12]:
valgen = resnet_chestxray.benchmark.valgen
STEP_SIZE_EVAL = len(valgen.index) // valgen.batch_size
print(len(valgen.index))
STEP_SIZE_EVAL // 10

27

In [13]:

eval_res = resnet_chestxray.model.evaluate(x=valgen, steps=STEP_SIZE_EVAL, verbose=1)


  ...
    to  
  ['...']
271/271 [==============================] - 75s 276ms/step - loss: 0.2603 - categorical_crossentropy: 0.2603 - categorical_accuracy: 0.2547 - categorical_hinge: 1.7244


In [22]:
[float (i) for i in eval_res]

[0.26028381849938104,
 0.26028382778167725,
 0.25472787022590637,
 1.7244476079940796]

In [13]:
predictions

NameError: name 'predictions' is not defined

In [12]:
print(resnet_chestxray.evaluation_result["report"])

                    precision    recall  f1-score   support

             Edema       0.00      0.00      0.00       455
       Atelectasis       0.07      0.00      0.00      2207
         Pneumonia       0.00      0.00      0.00       276
         Emphysema       0.00      0.00      0.00       466
      Pneumothorax       0.00      0.00      0.00      1018
Pleural_Thickening       0.00      0.00      0.00       654
      Cardiomegaly       0.00      0.00      0.00       546
      Infiltration       0.18      1.00      0.30      3981
     Consolidation       0.00      0.00      0.00       963
          Fibrosis       0.00      0.00      0.00       333
        No Finding       0.54      1.00      0.70     12130
          Effusion       0.26      0.18      0.21      2626
            Nodule       0.00      0.00      0.00      1351
              Mass       0.00      0.00      0.00      1257
            Hernia       0.00      0.00      0.00        52

         micro avg       0.36      0.5

In [13]:
import numpy as np
from sklearn.metrics import classification_report
classification_report

<function sklearn.metrics._classification.classification_report(y_true, y_pred, *, labels=None, target_names=None, sample_weight=None, digits=2, output_dict=False, zero_division='warn')>

In [33]:
y_pred = np.array([[1,0,0,0], [0,1,0,1], [1,1,0,1]])
y_true = np.array([[0,0,1,1], [0,1,1,0], [1,0,0,1]])
print(classification_report(y_true,y_pred))

              precision    recall  f1-score   support

           0       0.50      1.00      0.67         1
           1       0.50      1.00      0.67         1
           2       0.00      0.00      0.00         2
           3       0.50      0.50      0.50         2

   micro avg       0.50      0.50      0.50         6
   macro avg       0.38      0.62      0.46         6
weighted avg       0.33      0.50      0.39         6
 samples avg       0.39      0.50      0.43         6



In [1]:
mock_predictions = tf.constant([[0., 0., 0.5, 1.], [0.8, 0.1, 0.2, 0.1]])
mock_truth = tf.constant([[0., 0., 1., 1.], [1., 0., 1., 0.]])

tf.constant(1.) - mock_truth

NameError: name 'tf' is not defined

In [36]:
label_id = 0
single_pathology_pred = mock_predictions[:, label_id]
p = tf.stack([mock_predictions, 1. - mock_predictions], 1)
t = tf.stack([mock_truth, 1. - mock_truth], 1)
sum_bc=0
for i in range(4):
    sum_bc = tf.keras.losses.BinaryCrossentropy()(t[:, :, i], p[:, :, i])
sum_bc/4
t.dtype == tf.float32

True

In [38]:
def mean_bce(y_true, y_pred):
    print("y_true", y_true)
    print("y_pred", y_pred)
    if y_true.dtype == tf.float32 and y_pred.dtype == tf.float32:
        p = tf.stack([y_pred, tf.constant(1.) - y_pred], 1)
        t = tf.stack([y_true, tf.constant(1.) - y_true], 1)
        sum_bc=0
        for i in range(4):
            sum_bc = tf.keras.losses.BinaryCrossentropy()(t[:, :, i], p[:, :, i])
        return sum_bc/4
    else:
        print("WARNING: not type float32")
        return 100

In [63]:
y_true = [[0., 1.], [1., 0.]]
y_pred = [[1, 0], [1., 0]]
# Using 'auto'/'sum_over_batch_size' reduction type.
bce = tf.keras.losses.BinaryCrossentropy()
bce(y_true, y_pred).numpy()

7.6895466

In [6]:
import pandas as pd
from src.preprocessing.split.train_test_split import train_test_split
dataset_folder= Path(os.environ.get("CHEXPERT_DATASET_DIRECTORY")) 
train_valid_labels = pd.read_csv( dataset_folder/ "train.csv")
train_labels, validation_labels = train_test_split(train_valid_labels)

train_labels.head()

,Path,Sex,Age,Frontal/Lateral,AP/PA,No Finding,Enlarged Cardiomediastinum,Cardiomegaly,Lung Opacity,Lung Lesion,Edema,Consolidation,Pneumonia,Atelectasis,Pneumothorax,Pleural Effusion,Pleural Other,Fracture,Support Devices,patient_id
0,CheXpert-v1.0-small/train/patient00001/study1/...,Female,68,Frontal,AP,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,1.0,1
1,CheXpert-v1.0-small/train/patient00002/study2/...,Female,87,Frontal,AP,NaN,NaN,-1.0,1.0,NaN,-1.0,-1.0,NaN,-1.0,NaN,-1.0,NaN,1.0,NaN,2
2,CheXpert-v1.0-small/train/patient00002/study1/...,Female,83,Frontal,AP,NaN,NaN,NaN,1.0,NaN,NaN,-1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,2
3,CheXpert-v1.0-small/train/patient00002/study1/...,Female,83,Lateral,NaN,NaN,NaN,NaN,1.0,NaN,NaN,-1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,2
11,CheXpert-v1.0-small/train/patient00006/study1/...,Female,42,Frontal,AP,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,6


In [7]:

from src.datasets.generator import ImageDataGenerator
chexpert_columns = ['Enlarged Cardiomediastinum', 'Cardiomegaly', 'Lung Opacity']

train_generator = ImageDataGenerator(dataset=train_labels,
                               dataset_folder=dataset_folder,
                               label_columns=chexpert_columns,
                               batch_size=32)

eval_generator = ImageDataGenerator(dataset=validation_labels,
                               dataset_folder=dataset_folder,
                               label_columns=chexpert_columns,
                               batch_size=32)

In [8]:
train_data, train_data_labels = train_generator[0]
eval_data, eval_data_labels = eval_generator[0]

In [9]:
tf.version

<module 'tensorflow._api.v2.version' from '/usr/local/lib/python3.6/dist-packages/tensorflow/_api/v2/version/__init__.py'>

In [12]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import TensorBoard
from src.metrics.metrics import F2Score

metrics = [tf.keras.metrics.AUC(name="AUC_ROC", multi_label=True),
           tf.keras.metrics.Precision(),
           tf.keras.metrics.Recall(),
           F2Score(),
           tf.keras.metrics.BinaryAccuracy()
          ]
model_chexpert = SimpleBase(ResNet152V2, len(chexpert_columns)).get_model()
model_chexpert.compile(optimizer=Adam(),
                           loss= "binary_crossentropy",
                           metrics=metrics)

In [14]:

model_dir = Path("models/") / "resnet_test9"
log_dir = model_dir / "10"
log_dir.mkdir(parents=True, exist_ok=True)
tensorboard_callback = TensorBoard(log_dir=str(log_dir),
                                   update_freq= 5,
                                   histogram_freq=1,
                                   write_graph=False,
                                   embeddings_freq=1,
                                   profile_batch=0)
early_stopping_callback = tf.keras.callbacks.EarlyStopping(monitor='loss',
                                                           patience=3)
train_results = model_chexpert.fit(x=train_data,
                                   y=train_data_labels,
                                   steps_per_epoch=len(train_data),
                                   validation_data=(eval_data, eval_data_labels),
                                   validation_steps=len(eval_data),
                                   epochs=15,
                                   callbacks=[tensorboard_callback, early_stopping_callback])

Epoch 1/15
 4/32 [==>...........................] - ETA: 0s - loss: 0.0041 - AUC_ROC: 0.6667 - precision_2: 1.0000 - recall_2: 1.0000 - f2_score_1: 1.0000 - binary_accuracy: 1.0000    

NotFoundError: 2 root error(s) found.
  (0) Not found:  Resource localhost/_AnonymousVar1687/N10tensorflow22SummaryWriterInterfaceE does not exist.
	 [[{{node cond_5/then/_140/batch_loss}}]]
  (1) Not found:  Resource localhost/_AnonymousVar1687/N10tensorflow22SummaryWriterInterfaceE does not exist.
	 [[{{node cond_5/then/_140/batch_loss}}]]
	 [[assert_less_equal_4/Assert/AssertGuard/else/_123/assert_less_equal_4/Assert/AssertGuard/Assert/data_1/_218]]
0 successful operations.
0 derived errors ignored. [Op:__inference_train_function_55586]

Function call stack:
train_function -> train_function


In [48]:

y_pred = tf.constant([[0., 0., 0.5, 1.], [0.8, 0.1, 0.2, 0.1]])
y_true = tf.constant([[0., 0., 1., 1.], [1., 0., 1., 0.]])

epsilon = tf.constant(1e-07)
p_weight = tf.constant(0.5)
n_weight = tf.constant(0.5)
y_pred

<tf.Tensor: shape=(2, 4), dtype=float32, numpy=
array([[0. , 0. , 0.5, 1. ],
       [0.8, 0.1, 0.2, 0.1]], dtype=float32)>

In [50]:

bces = -(p_weight * y_true * tf.math.log(y_pred+epsilon) +
                   n_weight * (1 - y_true) * (1 - tf.math.log(y_pred) + epsilon))
bces

<tf.Tensor: shape=(2, 4), dtype=float32, numpy=
array([[          -inf,           -inf,  3.4657347e-01, -5.9604641e-08],
       [ 1.1157169e-01, -1.6512926e+00,  8.0471867e-01, -1.6512926e+00]],
      dtype=float32)>

In [54]:
p_weight * y_true * tf.math.log(y_pred+epsilon)

<tf.Tensor: shape=(2, 4), dtype=float32, numpy=
array([[-0.0000000e+00, -0.0000000e+00, -3.4657347e-01,  5.9604641e-08],
       [-1.1157169e-01, -0.0000000e+00, -8.0471867e-01, -0.0000000e+00]],
      dtype=float32)>

In [56]:
n_weight * (1 - y_true) * tf.math.log((1 - y_pred) + epsilon)

<tf.Tensor: shape=(2, 4), dtype=float32, numpy=
array([[ 5.9604641e-08,  5.9604641e-08, -0.0000000e+00, -0.0000000e+00],
       [-0.0000000e+00, -5.2680206e-02, -0.0000000e+00, -5.2680206e-02]],
      dtype=float32)>

In [57]:
tf.math.log(y_pred+epsilon)

<tf.Tensor: shape=(2, 4), dtype=float32, numpy=
array([[-1.6118095e+01, -1.6118095e+01, -6.9314694e-01,  1.1920928e-07],
       [-2.2314338e-01, -2.3025842e+00, -1.6094373e+00, -2.3025842e+00]],
      dtype=float32)>

In [98]:

y_pred = tf.constant([[0.2, 0.], [0.8, 0.1]])
y_true = tf.constant([[0., 0.], [1., 0.]])


p = tf.stack([y_pred, tf.constant(1.) - y_pred], 1)
t = tf.stack([y_true, tf.constant(1.) - y_true], 1)
for i in range(2):
    print("pred_"+str(i+1), "\n", y_pred[:,  i])
    print("true_"+str(i+1), "\n", y_true[:,  i])
    
print()
pred_1= tf.constant([[0.2, 0.8]])
true_1= tf.constant([[0., 1.]])

pred_2 = tf.constant([[0., 0.1]])
true_2 = tf.constant([[0., 0.]])

l1 = tf.keras.losses.BinaryCrossentropy()(true_1,pred_1)
l2 = tf.keras.losses.BinaryCrossentropy()(true_2,pred_2)
testl = tf.keras.losses.BinaryCrossentropy()(test_true,test_pred)
all_l = tf.keras.losses.BinaryCrossentropy()(y_true,y_pred)
print(l1)
print(l2)
print(testl)
print((l1 + l2)/2)
print(all_l)


pred_1 
 tf.Tensor([0.2 0.8], shape=(2,), dtype=float32)
true_1 
 tf.Tensor([0. 1.], shape=(2,), dtype=float32)
pred_2 
 tf.Tensor([0.  0.1], shape=(2,), dtype=float32)
true_2 
 tf.Tensor([0. 0.], shape=(2,), dtype=float32)

tf.Tensor(0.22314338, shape=(), dtype=float32)
tf.Tensor(0.052680206, shape=(), dtype=float32)
tf.Tensor(0.22314338, shape=(), dtype=float32)
tf.Tensor(0.1379118, shape=(), dtype=float32)
tf.Tensor(0.1379118, shape=(), dtype=float32)


In [90]:

from tensorflow.python.ops import math_ops
def custom_bce(y_t, y_p):
    bce = y_t * math_ops.log(y_p + epsilon)
    bce += (1 - y_t) * math_ops.log(1 - y_p + epsilon)
    return tf.reduce_mean(-bce)

print(custom_bce(true_1,pred_1))
print(custom_bce(true_2,pred_2))
print(custom_bce(test_true,test_pred))

tf.Tensor(0.22314338, shape=(), dtype=float32)
tf.Tensor(0.052680146, shape=(), dtype=float32)
tf.Tensor(0.22314338, shape=(), dtype=float32)


In [16]:
tf.constant(0.0)

<tf.Tensor: shape=(), dtype=float32, numpy=0.0>

In [21]:

from src.metrics.metrics import F2Score, MeanBinaryAccuracy
y_pred = tf.constant([[0.2, 0.3, 1.], [0.8, 0.1, 1.]])
y_true = tf.constant([[0., 1., 1.], [1., 0., 1.]])

TN = tf.keras.metrics.TrueNegatives()
_ = TN.update_state(y_true,y_pred)

TP = tf.keras.metrics.TruePositives()
_ = TP.update_state(y_true,y_pred)

print(TN.result().numpy())
print(TP.result().numpy())
print(tf.reduce_mean(tf.keras.metrics.binary_accuracy(y_true, y_pred)))

Prec = tf.keras.metrics.BinaryAccuracy()
_ = Prec.update_state(y_true,y_pred)
Prec.result().numpy()

2.0
3.0
tf.Tensor(0.8333334, shape=(), dtype=float32)


0.8333334

In [117]:
def multitask_accuracy ( y_true, y_pred, ):
    return tf.reduce_mean(tf.keras.metrics.binary_accuracy(y_true, y_pred, threshold=0.5))
multitask_accuracy(y_true, y_pred)

<tf.Tensor: shape=(), dtype=float32, numpy=0.8333334>

In [177]:

f2score = F2Score()
_ = f2score.update_state(y_true,y_pred)
f2score.result().numpy()

0.78947365